# Strong heterogeneity with DGP where PS and OR are incorrect

Treatment heterogeneity
- index_att = 10 * (z1 + z2 - z3 + z4)  # Strong heterogeneity in treatment effect
- This means that the treatment effect is a complex function of the covariates z1,z2,z3, and z4 Consequently, the effect of the treatment is not uniform across all individuals but varies depending on these covariates.
- Heterogeneous Treatment Effects: Variability in the treatment effect itself, based on measured covariates.

Not related is:
- unobserved heterogeneity of v = np.random.normal(index_lin, 1, n)
- This introduces variability in the outcomes that is correlated with the observed covariates.
- Unobserved heterogeneity: Variability in the outcome due to unmeasured factors

# Deep Learning Header

In [ ]:
# Deep Learning Header

import numpy as np
import statsmodels.api as sm
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

np.random.seed(42)
tf.random.set_seed(42)


class LossHistory(Callback):
    def on_train_begin(self, logs=None):
        if logs is None:
            logs = {}
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.losses.append(logs.get("loss"))
        self.val_losses.append(logs.get("val_loss"))

    def get_min_loss(self):
        return min(self.losses), min(self.val_losses)

# IPW DL heterogeneity

In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
from scipy.stats import iqr, norm
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

np.random.seed(42)


def std_ipw_did_rc(
    y,
    post,
    D,
    covariates=None,
    i_weights=None,
    boot=False,
    boot_type="weighted",
    nboot=None,
    inffunc=False,
):
    # Convert inputs to numpy arrays
    D = np.asarray(D).flatten()
    n = len(D)
    y = np.asarray(y).flatten()
    post = np.asarray(post).flatten()

    # Add constant to covariate vector
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.hstack((np.ones((n, 1)), covariates))

    # Weights
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        msg = "i.weights must be non-negative"
        raise ValueError(msg)

    def create_deep_ffnn(input_dim, depth, units, learning_rate, l2_reg):
        np.random.seed(42)

        inputs = Input(shape=(input_dim,))
        x = Dense(units, kernel_regularizer=l2(l2_reg))(inputs)
        x = ReLU()(x)

        for _ in range(depth - 2):
            x = Dense(units, kernel_regularizer=l2(l2_reg))(x)
            x = ReLU()(x)

        outputs = Dense(1, activation="sigmoid")(x)
        model = Model(inputs, outputs)

        # Compile the model with Adam optimizer and specified learning rate
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss="binary_crossentropy")

        return model

    # Assume `int_cov`, `D`, and `i_weights` are already defined as in your original code
    depth = 3
    units = 32
    learning_rate = 0.01
    l2_reg = 0.01
    input_dim = int_cov.shape[1]

    from sklearn.model_selection import train_test_split

    (
        int_cov_train,
        int_cov_val,
        D_train,
        D_val,
        weights_train,
        weights_val,
    ) = train_test_split(int_cov, D, i_weights, test_size=0.2, random_state=42)

    # Create and compile the model with the optimal hyperparameters
    model = create_deep_ffnn(input_dim, depth, units, learning_rate, l2_reg)
    history = LossHistory()

    # Train the model
    # Train the model
    model.fit(
        int_cov_train,
        D_train,
        sample_weight=weights_train,
        epochs=50,
        batch_size=32,
        verbose=0,
        validation_data=(int_cov_val, D_val, weights_val),
        callbacks=[history],
    )
    min_training_loss, min_validation_loss = history.get_min_loss()

    # Predict the probabilities
    ps_fit = model.predict(int_cov).flatten()

    # Ensure no values are exactly 0 or 1

    # Compute IPW estimator
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * y / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * y / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * y / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * y / np.mean(w_cont_post)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    # ATT estimator
    ipw_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre)

    # Get the influence function to compute standard error
    score_ps = i_weights.reshape(-1, 1) * (D - ps_fit).reshape(-1, 1) * int_cov
    hessian_ps = np.linalg.inv(np.dot(score_ps.T, score_ps) / n)
    asy_lin_rep_ps = np.dot(score_ps, hessian_ps)

    # Influence function of the "treat" component
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )
    inf_treat = inf_treat_post - inf_treat_pre

    # Influence function of the control component
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    # Estimation effect from gamma hat (pscore)
    M2_pre = np.mean(
        w_cont_pre.reshape(-1, 1)
        * (y - att_cont_pre).reshape(-1, 1)
        * int_cov
        / np.mean(w_cont_pre),
        axis=0,
    )
    M2_post = np.mean(
        w_cont_post.reshape(-1, 1)
        * (y - att_cont_post).reshape(-1, 1)
        * int_cov
        / np.mean(w_cont_post),
        axis=0,
    )

    inf_cont_ps = np.dot(asy_lin_rep_ps, (M2_post - M2_pre))
    inf_cont += inf_cont_ps

    # Influence function of the DR estimator
    att_inf_func = inf_treat - inf_cont

    if not boot:
        # Estimate standard error
        se_att = np.std(att_inf_func) / np.sqrt(n)
        uci = ipw_att + 1.96 * se_att
        lci = ipw_att - 1.96 * se_att
        ipw_boot = None
    else:
        if nboot is None:
            nboot = 999
        if boot_type == "multiplier":
            # Multiplier bootstrap
            multipliers = np.random.normal(size=(nboot, n))
            ipw_boot = [np.mean(m * att_inf_func) for m in multipliers]
            se_att = iqr(ipw_boot) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.percentile(np.abs(ipw_boot / se_att), 95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att
        else:
            # Weighted bootstrap
            ipw_boot = [
                wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights)
                for _ in range(nboot)
            ]
            se_att = iqr(ipw_boot - ipw_att) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.percentile(np.abs((ipw_boot - ipw_att) / se_att), 95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att

    if not inffunc:
        att_inf_func = None

    return {
        "ATT": ipw_att,
        "se": se_att,
        "uci": uci,
        "lci": lci,
        "boots": ipw_boot,
        "att_inf_func": att_inf_func,
        "min_training_loss": min_training_loss,
        "min_validation_loss": min_validation_loss,
    }


def wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights):
    boot_weights = np.random.choice(np.arange(1, n + 1), size=n, replace=True)
    return std_ipw_did_rc(y, post, D, int_cov, i_weights=boot_weights)["ATT"]


# Simulation setup
np.random.seed(42)


def ipw_sim_run(dgp_type):
    # Define parameters
    np.random.seed(42)  # You can use any integer value as the seed
    # Sample size
    n = 1000

    # pscore index (strength of common support)
    Xsi_ps = 0.75

    # Proportion in each period

    # Number of bootstrapped draws

    # Mean and Std deviation of Z's without truncation
    mean_z1 = np.exp(0.25 / 2)
    sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
    mean_z2 = 10
    sd_z2 = 0.54164
    mean_z3 = 0.21887
    sd_z3 = 0.04453
    mean_z4 = 402
    sd_z4 = 56.63891

    # Initialize empty lists to store results
    ATTE_estimates = []
    asymptotic_variance = []
    min_training_losses = []
    min_validation_losses = []
    coverage_indicators = []

    for _i in range(1000):
        # Gen covariates
        x1 = np.random.normal(0, 1, n)
        x2 = np.random.normal(0, 1, n)
        x3 = np.random.normal(0, 1, n)
        x4 = np.random.normal(0, 1, n)

        z1 = np.exp(x1 / 2)
        z2 = x2 / (1 + np.exp(x1)) + 10
        z3 = (x1 * x3 / 25 + 0.6) ** 3
        z4 = (x1 + x4 + 20) ** 2

        z1 = (z1 - mean_z1) / sd_z1
        z2 = (z2 - mean_z2) / sd_z2
        z3 = (z3 - mean_z3) / sd_z3
        z4 = (z4 - mean_z4) / sd_z4

        np.column_stack((x1, x2, x3, x4))
        np.column_stack((z1, z2, z3, z4))

        # Gen treatment groups
        # Propensity score
        pi = 1 / (1 + np.exp(-Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4)))
        d = np.random.rand(n) <= pi

        # Generate aux indexes for the potential outcomes
        index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

        # Create heterogeneous effects for the ATT, which is set approximately equal to zero
        index_unobs_het = d * index_lin
        # Create heterogeneous effects for the ATT
        index_att = 10 * (z1 + z2 - z3 + z4)  # Strong heterogeneity in treatment effect

        # This is the key for consistency of outcome regression
        index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

        # v is the unobserved heterogeneity
        v = np.random.normal(index_unobs_het, 1, n)

        # Gen realized outcome at time 0
        y00 = index_lin + v + np.random.normal(size=n)
        y10 = index_lin + v + np.random.normal(size=n)

        # Gen outcomes at time 1
        # First let's generate potential outcomes: y_1_potential
        y01 = index_lin + v + np.random.normal(size=n) + index_trend
        y11 = index_lin + v + np.random.normal(size=n) + index_trend + index_att

        # Generate "T"
        ti_nt = 0.5
        ti_t = 0.5
        ti = d * ti_t + (1 - d) * ti_nt
        post = np.random.rand(n) <= ti

        # Combine outcomes into panel data format
        y = np.where(
            d & post,
            y11,
            np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
        )

        # Gen id
        id_ = np.repeat(np.arange(1, n + 1), 2)
        time = np.tile([0, 1], n)

        # Put in a long data frame
        dta_long = pd.DataFrame(
            {
                "id": id_,
                "time": time,
                "y": np.tile(y, 2),
                "post": np.tile(post.astype(int), 2),
                "d": np.tile(d.astype(int), 2),
                "x1": np.tile(z1, 2),
                "x2": np.tile(z2, 2),
                "x3": np.tile(z3, 2),
                "x4": np.tile(z4, 2),
            },
        )
        dta_long["post:d"] = dta_long["post"] * dta_long["d"]
        dta_long = dta_long.sort_values(["id", "time"])

        # Run the IPW-DID estimator
        covariates = dta_long[["x1", "x2", "x3", "x4"]].values
        y = dta_long["y"].values
        post = dta_long["post"].values
        D = dta_long["d"].values

        result = std_ipw_did_rc(y, post, D, covariates)

        ATTE_estimates.append(result["ATT"])
        asymptotic_variance.append(result["se"] ** 2)
        min_training_losses.append(result["min_training_loss"])
        min_validation_losses.append(result["min_validation_loss"])

        # Calculate coverage indicator
        coverage_indicator = int(result["lci"] <= 0 <= result["uci"])
        coverage_indicators.append(coverage_indicator)
    # Calculate average bias, median bias, and RMSE
    true_ATT = 0

    # Bias calculations
    biases = np.array(ATTE_estimates) - true_ATT
    average_bias = np.mean(biases)
    median_bias = np.median(biases)
    average_variance = np.mean(asymptotic_variance)
    # RMSE calculation
    rmse = np.sqrt(np.mean(biases**2))
    avg_min_training_loss = np.mean(min_training_losses)
    avg_min_validation_loss = np.mean(min_validation_losses)
    avg_coverage_prob = np.mean(
        coverage_indicators,
    )  # Calculate average coverage probability

    results = {
        "avg_bias": average_bias,
        "med_bias": median_bias,
        "rmse": rmse,
        "average_variance": average_variance,
        "average_min_training_loss": avg_min_training_loss,
        "average_min_validation_loss": avg_min_validation_loss,
        "avg_coverage_prob": avg_coverage_prob,
    }

    # Ensure the directory exists
    os.makedirs("bld/het_results", exist_ok=True)
    latex_filename = f"bld/het_results/het_ipw_dl_{dgp_type}.tex"

    # Writing the results to a LaTeX file
    with open(latex_filename, "w") as f:
        f.write("\\begin{table}[ht]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{|l|r|}\n")
        f.write("\\hline\n")
        f.write("Metric & Value \\\\\n")
        f.write("\\hline\n")
        for key, value in results.items():
            f.write(f"{key.replace('_', ' ').title()} & {value:.4f} \\\\\n")
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write(
            f"\\caption{{Simulation Results for double robust deep learning with DGP Type {dgp_type}}}\n",
        )
        f.write("\\end{table}\n")

    # Save ATTE estimates as a pickle file
    os.makedirs("bld/het_results", exist_ok=True)
    pickle_filename = f"bld/het_results/het_ipw_dl_atte_estimates_dgp_{dgp_type}.pkl"
    with open(pickle_filename, "wb") as f:
        pickle.dump(ATTE_estimates, f)

    # Display the results
    return {
        "Average Bias": average_bias,
        "Median Bias": median_bias,
        "RMSE": rmse,
        "Average Variance of ATT": average_variance,
        "average_min_training_loss": avg_min_training_loss,
        "average_min_validation_loss": avg_min_validation_loss,
        "Average Coverage Probability": avg_coverage_prob,
    }


ipw_sim_run(dgp_type="4")

# DR DL NN Heterogeneity

In [ ]:
# Define the DRDID function
import pandas as pd
from tensorflow.keras.layers import Dense, Input, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


def create_deep_ffnn(input_dim, depth, units, learning_rate, l2_reg):
    np.random.seed(42)

    inputs = Input(shape=(input_dim,))
    x = Dense(units, kernel_regularizer=l2(l2_reg))(inputs)
    x = ReLU()(x)

    for _ in range(depth - 2):
        x = Dense(units, kernel_regularizer=l2(l2_reg))(x)
        x = ReLU()(x)

    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)

    # Compile the model with Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="binary_crossentropy")

    return model


def drdid_rc(y, post, D, covariates=None, i_weights=None):
    np.random.seed(42)

    # Ensure D is a vector
    D = np.asarray(D)
    # Sample size
    n = len(D)
    # Ensure y is a vector
    y = np.asarray(y)
    # Ensure post is a vector
    post = np.asarray(post)
    # Add constant to covariate vector
    int_cov = np.ones((n, 1))
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.hstack((np.ones((n, 1)), covariates))

    # Weights
    if i_weights is None:
        i_weights = np.ones(n)

    # Define parameters for the neural network
    depth = 3
    units = 32
    learning_rate = 0.01
    l2_reg = 0.01
    input_dim = int_cov.shape[1]

    # Split the data into training and validation sets
    from sklearn.model_selection import train_test_split

    (
        int_cov_train,
        int_cov_val,
        D_train,
        D_val,
        weights_train,
        weights_val,
    ) = train_test_split(int_cov, D, i_weights, test_size=0.2, random_state=42)

    # Create and compile the model with the optimal hyperparameters
    model = create_deep_ffnn(input_dim, depth, units, learning_rate, l2_reg)

    history = LossHistory()
    # Train the model with validation data
    model.fit(
        int_cov_train,
        D_train,
        sample_weight=weights_train,
        epochs=50,
        batch_size=32,
        verbose=0,
        validation_data=(int_cov_val, D_val, weights_val),
        callbacks=[history],
    )
    min_training_loss, min_validation_loss = history.get_min_loss()

    # Predict the probabilities
    ps_fit = model.predict(int_cov).flatten()

    # Ensure no values are exactly 0 or 1

    # Compute the Outcome regression for the control group at the pre-treatment period, using OLS
    reg_cont_pre = sm.WLS(
        y[(D == 0) & (post == 0)],
        int_cov[(D == 0) & (post == 0)],
        weights=i_weights[(D == 0) & (post == 0)],
    ).fit()
    out_y_cont_pre = int_cov @ reg_cont_pre.params

    # Compute the Outcome regression for the control group at the post-treatment period, using OLS
    reg_cont_post = sm.WLS(
        y[(D == 0) & (post == 1)],
        int_cov[(D == 0) & (post == 1)],
        weights=i_weights[(D == 0) & (post == 1)],
    ).fit()
    out_y_cont_post = int_cov @ reg_cont_post.params

    # Combine the ORs for control group
    out_y_cont = post * out_y_cont_post + (1 - post) * out_y_cont_pre

    # Compute the Outcome regression for the treated group at the pre-treatment period, using OLS
    reg_treat_pre = sm.WLS(
        y[(D == 1) & (post == 0)],
        int_cov[(D == 1) & (post == 0)],
        weights=i_weights[(D == 1) & (post == 0)],
    ).fit()
    out_y_treat_pre = int_cov @ reg_treat_pre.params

    # Compute the Outcome regression for the treated group at the post-treatment period, using OLS
    reg_treat_post = sm.WLS(
        y[(D == 1) & (post == 1)],
        int_cov[(D == 1) & (post == 1)],
        weights=i_weights[(D == 1) & (post == 1)],
    ).fit()
    out_y_treat_post = int_cov @ reg_treat_post.params

    # Weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    w_d = i_weights * D
    w_dt1 = i_weights * D * post
    w_dt0 = i_weights * D * (1 - post)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * (y - out_y_cont) / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * (y - out_y_cont) / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * (y - out_y_cont) / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * (y - out_y_cont) / np.mean(w_cont_post)

    # Extra elements for the locally efficient DRDID
    eta_d_post = w_d * (out_y_treat_post - out_y_cont_post) / np.mean(w_d)
    eta_dt1_post = w_dt1 * (out_y_treat_post - out_y_cont_post) / np.mean(w_dt1)
    eta_d_pre = w_d * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_d)
    eta_dt0_pre = w_dt0 * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_dt0)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    att_d_post = np.mean(eta_d_post)
    att_dt1_post = np.mean(eta_dt1_post)
    att_d_pre = np.mean(eta_d_pre)
    att_dt0_pre = np.mean(eta_dt0_pre)

    # ATT estimator
    dr_att = (
        (att_treat_post - att_treat_pre)
        - (att_cont_post - att_cont_pre)
        + (att_d_post - att_dt1_post)
        - (att_d_pre - att_dt0_pre)
    )

    # Get the influence function to compute standard error
    # Leading term of the influence function: no estimation effect
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )

    # Estimation effect from beta hat from post and pre-periods
    M1_post = -np.mean(
        w_treat_post[:, np.newaxis] * post[:, np.newaxis] * int_cov,
        axis=0,
    ) / np.mean(w_treat_post)
    M1_pre = -np.mean(
        w_treat_pre[:, np.newaxis] * (1 - post)[:, np.newaxis] * int_cov,
        axis=0,
    ) / np.mean(w_treat_pre)

    # Now get the influence function related to the estimation effect related to beta's
    inf_treat_or_post = np.dot(reg_cont_post.cov_params(), M1_post)
    inf_treat_or_pre = np.dot(reg_cont_pre.cov_params(), M1_pre)
    inf_treat_or = inf_treat_or_post + inf_treat_or_pre

    # Influence function for the treated component
    inf_treat = inf_treat_post - inf_treat_pre + np.sum(inf_treat_or)

    # Now, get the influence function of control component
    # Leading term of the influence function: no estimation effect from nuisance parameters
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)

    # Influence function for the control component
    inf_cont = inf_cont_post - inf_cont_pre

    # Get the influence function of the inefficient DR estimator (put all pieces together)
    dr_att_inf_func1 = inf_treat - inf_cont

    # Now, we only need to get the influence function of the adjustment terms
    # First, the terms as if all OR parameters were known
    inf_eff1 = eta_d_post - w_d * att_d_post / np.mean(w_d)
    inf_eff2 = eta_dt1_post - w_dt1 * att_dt1_post / np.mean(w_dt1)
    inf_eff3 = eta_d_pre - w_d * att_d_pre / np.mean(w_d)
    inf_eff4 = eta_dt0_pre - w_dt0 * att_dt0_pre / np.mean(w_dt0)
    inf_eff = (inf_eff1 - inf_eff2) - (inf_eff3 - inf_eff4)

    # Now the estimation effect of the OR coefficients
    mom_post = np.mean(
        (w_d / np.mean(w_d) - w_dt1 / np.mean(w_dt1))[:, np.newaxis] * int_cov,
        axis=0,
    )
    mom_pre = np.mean(
        (w_d / np.mean(w_d) - w_dt0 / np.mean(w_dt0))[:, np.newaxis] * int_cov,
        axis=0,
    )
    inf_or_post = np.dot(
        (reg_treat_post.cov_params() - reg_cont_post.cov_params()),
        mom_post,
    )
    inf_or_pre = np.dot(
        (reg_treat_pre.cov_params() - reg_cont_pre.cov_params()),
        mom_pre,
    )
    inf_or = inf_or_post - inf_or_pre
    inf_or = np.sum(inf_or)

    # Get the influence function of the locally efficient DR estimator (put all pieces together)
    dr_att_inf_func = dr_att_inf_func1 + inf_eff + inf_or

    # Estimate of standard error
    se_dr_att = np.std(dr_att_inf_func) / np.sqrt(n)

    uci = dr_att + 1.96 * se_dr_att  # Upper confidence interval
    lci = dr_att - 1.96 * se_dr_att  # Lower confidence interval
    return {
        "ATT": dr_att,
        "se": se_dr_att,
        "min_training_loss": min_training_loss,
        "min_validation_loss": min_validation_loss,
        "uci": uci,
        "lci": lci,
    }


# Define parameters


def sz_dl_dgp4(dgp_type):
    np.random.seed(42)  # You can use any integer value as the seed
    # Sample size
    n = 1000

    # pscore index (strength of common support)
    Xsi_ps = 0.75

    # Proportion in each period

    # Number of bootstrapped draws

    # Mean and Std deviation of Z's without truncation
    mean_z1 = np.exp(0.25 / 2)
    sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
    mean_z2 = 10
    sd_z2 = 0.54164
    mean_z3 = 0.21887
    sd_z3 = 0.04453
    mean_z4 = 402
    sd_z4 = 56.63891

    # Initialize empty lists to store results
    ATTE_estimates = []
    asymptotic_variance = []
    min_training_losses = []
    min_validation_losses = []
    coverage_indicators = []
    individual_coverage_probs = []
    for _i in range(1000):
        # Gen covariates
        x1 = np.random.normal(0, 1, n)
        x2 = np.random.normal(0, 1, n)
        x3 = np.random.normal(0, 1, n)
        x4 = np.random.normal(0, 1, n)

        z1 = np.exp(x1 / 2)
        z2 = x2 / (1 + np.exp(x1)) + 10
        z3 = (x1 * x3 / 25 + 0.6) ** 3
        z4 = (x1 + x4 + 20) ** 2

        z1 = (z1 - mean_z1) / sd_z1
        z2 = (z2 - mean_z2) / sd_z2
        z3 = (z3 - mean_z3) / sd_z3
        z4 = (z4 - mean_z4) / sd_z4

        np.column_stack((x1, x2, x3, x4))
        np.column_stack((z1, z2, z3, z4))

        # Gen treatment groups
        # Propensity score
        pi = 1 / (1 + np.exp(-Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4)))
        d = np.random.rand(n) <= pi

        # Generate aux indexes for the potential outcomes
        index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

        # Create heterogeneous effects for the ATT, which is set approximately equal to zero
        index_unobs_het = d * index_lin
        index_att = 0

        # This is the key for consistency of outcome regression
        index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

        # Create heterogeneous effects for the ATT
        index_att = 10 * (z1 + z2 - z3 + z4)  # Strong heterogeneity in treatment effect
        # v is the unobserved heterogeneity
        v = np.random.normal(index_unobs_het, 1, n)

        # Gen realized outcome at time 0
        y00 = index_lin + v + np.random.normal(size=n)
        y10 = index_lin + v + np.random.normal(size=n)

        # Gen outcomes at time 1
        # First let's generate potential outcomes: y_1_potential
        y01 = index_lin + v + np.random.normal(size=n) + index_trend
        y11 = index_lin + v + np.random.normal(size=n) + index_trend + index_att

        # Generate "T"
        ti_nt = 0.5
        ti_t = 0.5
        ti = d * ti_t + (1 - d) * ti_nt
        post = np.random.rand(n) <= ti

        # Combine outcomes into panel data format
        y = np.where(
            d & post,
            y11,
            np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
        )

        # Gen id
        id_ = np.repeat(np.arange(1, n + 1), 2)
        time = np.tile([0, 1], n)

        # Put in a long data frame
        dta_long = pd.DataFrame(
            {
                "id": id_,
                "time": time,
                "y": np.tile(y, 2),
                "post": np.tile(post.astype(int), 2),
                "d": np.tile(d.astype(int), 2),
                "x1": np.tile(z1, 2),
                "x2": np.tile(z2, 2),
                "x3": np.tile(z3, 2),
                "x4": np.tile(z4, 2),
            },
        )
        dta_long["post:d"] = dta_long["post"] * dta_long["d"]
        dta_long = dta_long.sort_values(["id", "time"])

        # Run the IPW-DID estimator
        covariates = dta_long[["x1", "x2", "x3", "x4"]].values
        y = dta_long["y"].values
        post = dta_long["post"].values
        D = dta_long["d"].values

        result = drdid_rc(y, post, D, covariates)

        ATTE_estimates.append(result["ATT"])
        asymptotic_variance.append(result["se"] ** 2)
        min_training_losses.append(result["min_training_loss"])
        min_validation_losses.append(result["min_validation_loss"])

        coverage_indicator = int(result["lci"] <= 0 <= result["uci"])
        coverage_indicators.append(coverage_indicator)
        individual_coverage_probs.append(coverage_indicator)

    # Calculate average bias, median bias, and RMSE
    true_ATT = 0
    average_bias = np.mean(ATTE_estimates) - true_ATT
    median_bias = np.median(ATTE_estimates) - true_ATT
    rmse = np.sqrt(np.mean((np.array(ATTE_estimates) - true_ATT) ** 2))

    avg_min_training_loss = np.mean(min_training_losses)
    avg_min_validation_loss = np.mean(min_validation_losses)
    avg_coverage_prob = np.mean(
        coverage_indicators,
    )  # Calculate average coverage probability

    # Calculate average of the variance
    average_variance = np.mean(asymptotic_variance)
    results = {
        "avg_bias": average_bias,
        "med_bias": median_bias,
        "rmse": rmse,
        "average_variance": average_variance,
        "average_min_training_loss": avg_min_training_loss,
        "average_min_validation_loss": avg_min_validation_loss,
        "avg_coverage_prob": avg_coverage_prob,  # Include average coverage probability
    }
    # Ensure the directory exists
    # Ensure the directory exists
    os.makedirs("bld/het_results", exist_ok=True)
    latex_filename = f"bld/het_results/het_dr_dl_{dgp_type}.tex"

    # Writing the results to a LaTeX file
    with open(latex_filename, "w") as f:
        f.write("\\begin{table}[ht]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{|l|r|}\n")
        f.write("\\hline\n")
        f.write("Metric & Value \\\\\n")
        f.write("\\hline\n")
        for key, value in results.items():
            f.write(f"{key.replace('_', ' ').title()} & {value:.4f} \\\\\n")
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write(
            f"\\caption{{Simulation Results for double robust deep learning with DGP Type {dgp_type}}}\n",
        )
        f.write("\\end{table}\n")

    # Save ATTE estimates as a pickle file
    os.makedirs("bld/het_results", exist_ok=True)
    pickle_filename = f"bld/het_results/het_dr_dl_{dgp_type}.pkl"
    with open(pickle_filename, "wb") as f:
        pickle.dump(ATTE_estimates, f)

    return {
        "Average Bias": average_bias,
        "Median Bias": median_bias,
        "RMSE": rmse,
        "Average Variance of ATT": average_variance,
        "average_min_training_loss": avg_min_training_loss,
        "average_min_validation_loss": avg_min_validation_loss,
        "Average Coverage Probability": avg_coverage_prob,
    }


sz_dl_dgp4(dgp_type="4")

# Diagnosis Probability Bonds

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, Input, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


class LossHistory(Callback):
    def on_train_begin(self, logs=None):
        if logs is None:
            logs = {}
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.losses.append(logs.get("loss"))
        self.val_losses.append(logs.get("val_loss"))

    def get_min_loss(self):
        return min(self.losses), min(self.val_losses)


def create_deep_ffnn(input_dim, depth, units, learning_rate, l2_reg):
    np.random.seed(42)

    inputs = Input(shape=(input_dim,))
    x = Dense(units, kernel_regularizer=l2(l2_reg))(inputs)
    x = ReLU()(x)

    for _ in range(depth - 2):
        x = Dense(units, kernel_regularizer=l2(l2_reg))(x)
        x = ReLU()(x)

    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)

    # Compile the model with Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="binary_crossentropy")

    return model


def drdid_rc(y, post, D, covariates=None, i_weights=None):
    np.random.seed(42)

    # Ensure D is a vector
    D = np.asarray(D)
    # Sample size
    n = len(D)
    # Ensure y is a vector
    y = np.asarray(y)
    # Ensure post is a vector
    post = np.asarray(post)
    # Add constant to covariate vector
    int_cov = np.ones((n, 1))
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.hstack((np.ones((n, 1)), covariates))

    # Weights
    if i_weights is None:
        i_weights = np.ones(n)

    # Define parameters for the neural network
    depth = 3
    units = 32
    learning_rate = 0.01
    l2_reg = 0.01
    input_dim = int_cov.shape[1]

    # Split the data into training and validation sets
    (
        int_cov_train,
        int_cov_val,
        D_train,
        D_val,
        weights_train,
        weights_val,
    ) = train_test_split(int_cov, D, i_weights, test_size=0.2, random_state=42)

    # Create and compile the model with the optimal hyperparameters
    model = create_deep_ffnn(input_dim, depth, units, learning_rate, l2_reg)

    history = LossHistory()
    # Train the model with validation data
    model.fit(
        int_cov_train,
        D_train,
        sample_weight=weights_train,
        epochs=50,
        batch_size=32,
        verbose=0,
        validation_data=(int_cov_val, D_val, weights_val),
        callbacks=[history],
    )
    min_training_loss, min_validation_loss = history.get_min_loss()

    # Predict the probabilities
    ps_fit = model.predict(int_cov).flatten()

    # Ensure no values are exactly 0 or 1

    # Compute the Outcome regression for the control group at the pre-treatment period, using OLS
    reg_cont_pre = sm.WLS(
        y[(D == 0) & (post == 0)],
        int_cov[(D == 0) & (post == 0)],
        weights=i_weights[(D == 0) & (post == 0)],
    ).fit()
    out_y_cont_pre = int_cov @ reg_cont_pre.params

    # Compute the Outcome regression for the control group at the post-treatment period, using OLS
    reg_cont_post = sm.WLS(
        y[(D == 0) & (post == 1)],
        int_cov[(D == 0) & (post == 1)],
        weights=i_weights[(D == 0) & (post == 1)],
    ).fit()
    out_y_cont_post = int_cov @ reg_cont_post.params

    # Combine the ORs for control group
    out_y_cont = post * out_y_cont_post + (1 - post) * out_y_cont_pre

    # Compute the Outcome regression for the treated group at the pre-treatment period, using OLS
    reg_treat_pre = sm.WLS(
        y[(D == 1) & (post == 0)],
        int_cov[(D == 1) & (post == 0)],
        weights=i_weights[(D == 1) & (post == 0)],
    ).fit()
    out_y_treat_pre = int_cov @ reg_treat_pre.params

    # Compute the Outcome regression for the treated group at the post-treatment period, using OLS
    reg_treat_post = sm.WLS(
        y[(D == 1) & (post == 1)],
        int_cov[(D == 1) & (post == 1)],
        weights=i_weights[(D == 1) & (post == 1)],
    ).fit()
    out_y_treat_post = int_cov @ reg_treat_post.params

    # Weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    w_d = i_weights * D
    w_dt1 = i_weights * D * post
    w_dt0 = i_weights * D * (1 - post)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * (y - out_y_cont) / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * (y - out_y_cont) / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * (y - out_y_cont) / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * (y - out_y_cont) / np.mean(w_cont_post)

    # Extra elements for the locally efficient DRDID
    eta_d_post = w_d * (out_y_treat_post - out_y_cont_post) / np.mean(w_d)
    eta_dt1_post = w_dt1 * (out_y_treat_post - out_y_cont_post) / np.mean(w_dt1)
    eta_d_pre = w_d * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_d)
    eta_dt0_pre = w_dt0 * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_dt0)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    att_d_post = np.mean(eta_d_post)
    att_dt1_post = np.mean(eta_dt1_post)
    att_d_pre = np.mean(eta_d_pre)
    att_dt0_pre = np.mean(eta_dt0_pre)

    # ATT estimator
    dr_att = (
        (att_treat_post - att_treat_pre)
        - (att_cont_post - att_cont_pre)
        + (att_d_post - att_dt1_post)
        - (att_d_pre - att_dt0_pre)
    )

    # Get the influence function to compute standard error
    # Leading term of the influence function: no estimation effect
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )

    # Estimation effect from beta hat from post and pre-periods
    M1_post = -np.mean(
        w_treat_post[:, np.newaxis] * post[:, np.newaxis] * int_cov,
        axis=0,
    ) / np.mean(w_treat_post)
    M1_pre = -np.mean(
        w_treat_pre[:, np.newaxis] * (1 - post)[:, np.newaxis] * int_cov,
        axis=0,
    ) / np.mean(w_treat_pre)

    # Now get the influence function related to the estimation effect related to beta's
    inf_treat_or_post = np.dot(reg_cont_post.cov_params(), M1_post)
    inf_treat_or_pre = np.dot(reg_cont_pre.cov_params(), M1_pre)
    inf_treat_or = inf_treat_or_post + inf_treat_or_pre

    # Influence function for the treated component
    inf_treat = inf_treat_post - inf_treat_pre + np.sum(inf_treat_or)

    # Now, get the influence function of control component
    # Leading term of the influence function: no estimation effect from nuisance parameters
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)

    # Influence function for the control component
    inf_cont = inf_cont_post - inf_cont_pre

    # Get the influence function of the inefficient DR estimator (put all pieces together)
    dr_att_inf_func1 = inf_treat - inf_cont

    # Now, we only need to get the influence function of the adjustment terms
    # First, the terms as if all OR parameters were known
    inf_eff1 = eta_d_post - w_d * att_d_post / np.mean(w_d)
    inf_eff2 = eta_dt1_post - w_dt1 * att_dt1_post / np.mean(w_dt1)
    inf_eff3 = eta_d_pre - w_d * att_d_pre / np.mean(w_d)
    inf_eff4 = eta_dt0_pre - w_dt0 * att_dt0_pre / np.mean(w_dt0)
    inf_eff = (inf_eff1 - inf_eff2) - (inf_eff3 - inf_eff4)

    # Now the estimation effect of the OR coefficients
    mom_post = np.mean(
        (w_d / np.mean(w_d) - w_dt1 / np.mean(w_dt1))[:, np.newaxis] * int_cov,
        axis=0,
    )
    mom_pre = np.mean(
        (w_d / np.mean(w_d) - w_dt0 / np.mean(w_dt0))[:, np.newaxis] * int_cov,
        axis=0,
    )
    inf_or_post = np.dot(
        (reg_treat_post.cov_params() - reg_cont_post.cov_params()),
        mom_post,
    )
    inf_or_pre = np.dot(
        (reg_treat_pre.cov_params() - reg_cont_pre.cov_params()),
        mom_pre,
    )
    inf_or = inf_or_post - inf_or_pre
    inf_or = np.sum(inf_or)

    # Get the influence function of the locally efficient DR estimator (put all pieces together)
    dr_att_inf_func = dr_att_inf_func1 + inf_eff + inf_or

    # Estimate of standard error
    se_dr_att = np.std(dr_att_inf_func) / np.sqrt(n)

    uci = dr_att + 1.96 * se_dr_att  # Upper confidence interval
    lci = dr_att - 1.96 * se_dr_att  # Lower confidence interval
    return {
        "ATT": dr_att,
        "se": se_dr_att,
        "min_training_loss": min_training_loss,
        "min_validation_loss": min_validation_loss,
        "uci": uci,
        "lci": lci,
    }


# Define parameters


def sz_dl_dgp4(dgp_type):
    np.random.seed(42)  # You can use any integer value as the seed
    # Sample size
    n = 1000

    # pscore index (strength of common support)
    Xsi_ps = 0.75

    # Proportion in each period

    # Number of bootstrapped draws

    # Mean and Std deviation of Z's without truncation
    mean_z1 = np.exp(0.25 / 2)
    sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
    mean_z2 = 10
    sd_z2 = 0.54164
    mean_z3 = 0.21887
    sd_z3 = 0.04453
    mean_z4 = 402
    sd_z4 = 56.63891

    # Initialize empty lists to store results
    ATTE_estimates = []
    asymptotic_variance = []
    min_training_losses = []
    min_validation_losses = []
    coverage_indicators = []
    individual_coverage_probs = (
        []
    )  # New list to store individual coverage probabilities
    ci_bounds = []
    for _i in range(1000):
        # Gen covariates
        x1 = np.random.normal(0, 1, n)
        x2 = np.random.normal(0, 1, n)
        x3 = np.random.normal(0, 1, n)
        x4 = np.random.normal(0, 1, n)

        z1 = np.exp(x1 / 2)
        z2 = x2 / (1 + np.exp(x1)) + 10
        z3 = (x1 * x3 / 25 + 0.6) ** 3
        z4 = (x1 + x4 + 20) ** 2

        z1 = (z1 - mean_z1) / sd_z1
        z2 = (z2 - mean_z2) / sd_z2
        z3 = (z3 - mean_z3) / sd_z3
        z4 = (z4 - mean_z4) / sd_z4

        np.column_stack((x1, x2, x3, x4))
        np.column_stack((z1, z2, z3, z4))

        # Gen treatment groups
        # Propensity score
        pi = 1 / (1 + np.exp(-Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4)))
        d = np.random.rand(n) <= pi

        # Generate aux indexes for the potential outcomes
        index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

        # Create heterogeneous effects for the ATT, which is set approximately equal to zero
        index_unobs_het = d * index_lin
        index_att = 0

        # This is the key for consistency of outcome regression
        index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

        # Create heterogeneous effects for the ATT
        index_att = 10 * (z1 + z2 - z3 + z4)  # Strong heterogeneity in treatment effect
        # v is the unobserved heterogeneity
        v = np.random.normal(index_unobs_het, 1, n)

        # Gen realized outcome at time 0
        y00 = index_lin + v + np.random.normal(size=n)
        y10 = index_lin + v + np.random.normal(size=n)

        # Gen outcomes at time 1
        # First let's generate potential outcomes: y_1_potential
        y01 = index_lin + v + np.random.normal(size=n) + index_trend
        y11 = index_lin + v + np.random.normal(size=n) + index_trend + index_att

        # Generate "T"
        ti_nt = 0.5
        ti_t = 0.5
        ti = d * ti_t + (1 - d) * ti_nt
        post = np.random.rand(n) <= ti

        # Combine outcomes into panel data format
        y = np.where(
            d & post,
            y11,
            np.where(~d & post, y01, np.where(~d & ~post, y00, y10)),
        )

        # Gen id
        id_ = np.repeat(np.arange(1, n + 1), 2)
        time = np.tile([0, 1], n)

        # Put in a long data frame
        dta_long = pd.DataFrame(
            {
                "id": id_,
                "time": time,
                "y": np.tile(y, 2),
                "post": np.tile(post.astype(int), 2),
                "d": np.tile(d.astype(int), 2),
                "x1": np.tile(z1, 2),
                "x2": np.tile(z2, 2),
                "x3": np.tile(z3, 2),
                "x4": np.tile(z4, 2),
            },
        )
        dta_long["post:d"] = dta_long["post"] * dta_long["d"]
        dta_long = dta_long.sort_values(["id", "time"])

        # Run the IPW-DID estimator
        covariates = dta_long[["x1", "x2", "x3", "x4"]].values
        y = dta_long["y"].values
        post = dta_long["post"].values
        D = dta_long["d"].values

        result = drdid_rc(y, post, D, covariates)

        ATTE_estimates.append(result["ATT"])
        asymptotic_variance.append(result["se"] ** 2)
        min_training_losses.append(result["min_training_loss"])
        min_validation_losses.append(result["min_validation_loss"])

        # Calculate coverage indicator
        coverage_indicator = int(result["lci"] <= 0 <= result["uci"])
        coverage_indicators.append(coverage_indicator)

        # Calculate individual coverage probability
        individual_coverage_probs.append(coverage_indicator)
        ci_bounds.append((result["lci"], result["uci"]))

    # Calculate average bias, median bias, and RMSE
    true_ATT = 0
    average_bias = np.mean(ATTE_estimates) - true_ATT
    median_bias = np.median(ATTE_estimates) - true_ATT
    rmse = np.sqrt(np.mean((np.array(ATTE_estimates) - true_ATT) ** 2))

    avg_min_training_loss = np.mean(min_training_losses)
    avg_min_validation_loss = np.mean(min_validation_losses)
    avg_coverage_prob = np.mean(
        coverage_indicators,
    )  # Calculate average coverage probability
    avg_coverage_prob_indiv = np.mean(individual_coverage_probs)
    # Calculate average of the variance
    average_variance = np.mean(asymptotic_variance)

    # Additional analysis for variability
    std_dev = np.std(ATTE_estimates)
    range_values = np.ptp(ATTE_estimates)
    iqr = np.percentile(ATTE_estimates, 75) - np.percentile(ATTE_estimates, 25)

    # Calculate average confidence interval bounds
    avg_lci = np.mean([bounds[0] for bounds in ci_bounds])
    avg_uci = np.mean([bounds[1] for bounds in ci_bounds])

    # Plot histogram
    plt.figure(figsize=(10, 6))
    sns.kdeplot(
        ATTE_estimates,
        fill=True,
        color="black",
        bw_adjust=0.5,
        alpha=0.3,
    )  # Adjust bw_adjust to control smoothness
    plt.axvline(x=avg_lci, color="black", linestyle="--")
    plt.axvline(x=avg_uci, color="black", linestyle="--")
    plt.xlabel("Conditional Average Treatment on the Treated Effect")
    plt.ylabel("Density")
    plt.legend()
    plt.savefig("./paper/graphs/atte_bounds.png")
    plt.show()

    return {
        "Average Bias": average_bias,
        "Median Bias": median_bias,
        "RMSE": rmse,
        "Average Variance of ATT": average_variance,
        "Average Min Training Loss": avg_min_training_loss,
        "Average Min Validation Loss": avg_min_validation_loss,
        "Average Coverage Probability": avg_coverage_prob,
        "Individual Coverage Probs": avg_coverage_prob_indiv,
        "Standard Deviation": std_dev,
        "Range": range_values,
        "IQR": iqr,
        "CI Bounds": ci_bounds,  # Return the confidence interval bounds
        "Average Lower CI": avg_lci,
        "Average Upper CI": avg_uci,
    }


# Run the simulation
results = sz_dl_dgp4(dgp_type="4")
print(results)
print("Confidence Interval Bounds for Each Run:")
for i, bounds in enumerate(results["CI Bounds"], start=1):
    print(f"Run {i}: Lower CI = {bounds[0]}, Upper CI = {bounds[1]}")